# Accessing larlite Data Products (C++ Classes and Their Functions) In Python
#### Note: you need to "source config/setup.sh" from your local larlite install in your terminal before typing "jupyter notebook blahblah.ipynb" for this to work. I haven't been able to effectively source this from within a jupyter notebook.

In [1]:
# These imports are actually NOT NEEDED if you already know the name of
# the ttree within the root file.
from ROOT import TFile, TTree

#import numpy as np
#import matplotlib.pylab as plt
import pandas as pd

In [2]:
# This was my attempt to source config/setup.sh that doesn't end up working
#!source /Users/davidkaleko/larlite/config/setup.sh

In [3]:
# Sample "litescanned" file that I happen to have lying around
myfile =  '/Users/davidkaleko/Data/larlite/mcc7_022316/'
myfile += 'scan_prod_pi0_0-1.0GeV_isotropic_v1/larlite_mcinfo_all.root'

In [4]:
f = TFile(myfile,"READ")
# Let's look at what's in the file:
print "Things stored in this file are:"
print [mykey.GetName() for mykey in f.GetListOfKeys()]
print
print "# of entries (# of events) stored in the mcshower tree is %d." % \
    f.mcshower_mcreco_tree.GetEntries()

Things stored in this file are:
['larlite_id_tree', 'gtruth_generator_tree', 'mctruth_corsika_tree', 'mctruth_generator_tree', 'mcflux_generator_tree', 'mcshower_mcreco_tree', 'mctrack_mcreco_tree', 'potsummary_generator_tree']

# of entries (# of events) stored in the mcshower tree is 5050.


#### This is almost certainly NOT the best way to do this but it's how I currently do things. Here I loop over the mcshower ttree, and for each event (each TTree entry) I require there are exactly two mcshowers. For those events with 2 mcshowers, I store each shower's deposited energy, then put those into a dataframe with two columns for later analysis.

In [5]:
mydata = []
for ientry in xrange(f.mcshower_mcreco_tree.GetEntries()):
    # You must do a "TTree.GetEntry" for each event
    blah = f.mcshower_mcreco_tree.GetEntry(ientry)
    # The ttree's "branch" with the same name is exactly what
    # you are used to seeing as "event_mcshower" in larlite analysis code
    ev_mcs = f.mcshower_mcreco_tree.mcshower_mcreco_branch
    # The size of ev_mcshower is the number of mcshowers in this specific event.
    if ev_mcs.size() != 2: continue
    # There are many ways to make a dataframe, but one I use is to create a
    # list of tuples. The length of the list will be the number of rows in the
    # resulting dataframe, and the length of each tuple in the list will be
    # the number of columns in the resulting dataframe.
    mydata.append( ( ev_mcs.at(0).DetProfile().E(), ev_mcs.at(1).DetProfile().E() ) )

In [6]:
# This is the dataframe function that turns the above list-of-tuples into a dataframe.
# Again there are probably better ways to do this (maybe that don't even involve a
# for-loop!!)
df = pd.DataFrame.from_records(mydata)

# Here I'm just naming the columns for my convenience.
df.columns = ['Shower1Energy', 'Shower2Energy']

In [7]:
df.head()

,Shower1Energy,Shower2Energy
0,145.217978,356.169564
1,0.000000,90.938522
2,0.888849,696.780593
3,583.193069,110.480748
4,29.060985,115.345036
